# Import model to Vertex AI
This notebook-based tutorial will create a example Xgboost model and deploy model on Vertex AI.

It contains:

- 1. Create a Xgboost model
- 2. Upload model to GCS
- 3. Import model to Vertex AI and deploy

# Setting up

In [ ]:
# !pip install xgboost

## 1. Create a Xgboost model with digits datasets

In [ ]:
from sklearn import datasets
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# load datasets
digits = datasets.load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model training
model = XGBClassifier()
model.fit(X_train, y_train)

# predict score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Save model artifact to local filesystem (doesn't persist)
artifact_filename = 'model.bst'
local_path = artifact_filename
model.save_model(local_path)

## 2. Upload model to GCS

Setting up bucket

In [ ]:
bucket_name = ''
gcs = 'gs://' + bucket_name
model_directory = f'{gcs}/' + 'models' + '/' + 'xgboost'

Create a bucket to store model

In [ ]:
!gsutil mb -l us-central1 $bucket_name

Upload model artifact to Cloud Storage

In [ ]:
from google.cloud import storage
import os

storage_path = os.path.join(model_directory, artifact_filename)
blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())
blob.upload_from_filename(local_path)

## 3. Import model to Vertex AI and deploy

In [ ]:
project = ''
location = 'us-central1'
display_name = 'digits-xgboost-v1'
serving_container_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-4:latest'

Import model to Vertex AI

In [ ]:
from utils import upload_model_sample

model = upload_model_sample.upload_model_sample(
    project=project,
    location=location,
    display_name=display_name,
    serving_container_image_uri=serving_container_image_uri,
    artifact_uri=model_directory
)

Check ['Vertex AI > Models'](https://console.cloud.google.com/vertex-ai/models)

In [ ]:
deployment = model.deploy(
    deployed_model_display_name = 'serving-digits-xgboost-v1',
    machine_type = 'n1-standard-2',
)

Check ['Vertex AI > Endpoints'](https://console.cloud.google.com/vertex-ai/endpoints)

# Clean up

To clean up all Google Cloud resource used in this project

In [ ]:
deployment.undeploy_all()
deployment.delete()
!gsutil rm -rf gs://${model_directory}